# Regularisierung auf dem Titanic-Datensatz

Hinweis: Da das Notebook nur das Prinizip der Regularisierung zeigen soll, ist der ML-Worflow in diesem Notebook stark vereinfacht (d.h. kein Auffüllen von N/As, kein Feature-Scaling und keine kategorischen Features).

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, Ridge, RidgeCV, Lasso, LassoCV
from sklearn.metrics import accuracy_score


pd.options.mode.chained_assignment = None  # avoid slide-copy-warning 

In [2]:
df = pd.read_csv("data/titanic.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
df_selection = df[["Survived", "Pclass", "Age", "SibSp", "Parch", "Fare"]]
df_selection = df_selection.dropna()
df_selection.head()

,Survived,Pclass,Age,SibSp,Parch,Fare
0,0,3,22.0,1,0,7.2500
1,1,1,38.0,1,0,71.2833
2,1,3,26.0,0,0,7.9250
3,1,1,35.0,1,0,53.1000
4,0,3,35.0,0,0,8.0500


In [4]:
df_X = df_selection.drop(columns = ["Survived"])
df_y = df_selection["Survived"]
X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.2, random_state=0)

## Aufgaben:

### Aufgabe 1: Ridge-Regularisierung
1. Trainieren Sie eine Logistische Regression mit Ridge-Regularisierung (siehe [RidgeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeClassifier.html)) mit $\alpha =100$ auf den Trainingsdaten.
2. Bestimmen Sie die Accuracy auf den Testdaten.

In [5]:
ridge = Ridge(alpha = 100)
ridge.fit(X_train, y_train)

Ridge(alpha=100)

In [6]:
# predict the test set
y_pred = ridge.predict(X_test)
# calculate the accuracy
accuracy = accuracy_score(y_test, y_pred.round())
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 67.13%


### Aufgabe 2: Cross-Validation über $\alpha$
1. Führen Sie nun eine Cross-Validation mit Hilfe der Klasse [RidgeClassifierCV](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeClassifierCV.html) durch. Benutzen Sie dafür die gleichen Werte für $\alpha$ wie im Notebook [`4_OverUnderfitting.ipynb`](4_OverUnderfitting.ipynb) in Zelle 9.
2. Bestimmen Sie die Accuracy auf den Testdaten.

In [7]:
alphas = list(range(1, 10000))
ridgecv = RidgeCV(alphas=alphas, scoring='neg_mean_squared_error')
ridgecv.fit(X_train, y_train)
ridgecv.alpha_ # bester Parameter
# predict the test set with the best parameter
y_pred = ridgecv.predict(X_test)
# calculate the accuracy
accuracy = accuracy_score(y_test, y_pred.round())
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 68.53%


### Aufgabe 3:  Lasso-Regularisierung
Für die Lasso-Regularisierung gibt es keine extra Unterklasse, da diese bei der Klasse `LogisitcRegression` über den Parameter `penalty` eingestellt werden kann. Mit dem Parameter `penalty = "l1"` wird die Lasso-Regression verwendet.
Der Regularisierungsfaktor $\alpha$ wird in diesem Fall über den Parameter `C` bestimmt, welcher den Default-Wert `C=1.0` hat:

`LogisticRegression(max_iter=1000, penalty = "l1", C=1.0, solver="liblinear")`

Beachten Sie dabei:
- Für die Regression ist das zusätzliche Argument `solver="liblinear"` nötig, da der Standard-Optimierer nicht mit `L1` funktioniert.
- Parameter `C` gibt die <ins>inverse</ins> Stärke der Regularisierung an (Details siehe [hier](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)).


1. Trainieren Sie eine Logistische Regression mit Lasso-Regression für verschiedene Parameter `C`:
    - den Default-Wert `C=1.0`.
    - wählen Sie `C` so, dass das Modell sehr stark regularisiert ist.
    - wählen Sie `C` so, dass das Modell keine Regularisierung hat.
2. Vergleiche Sie die Ergebnisse dieser Modelle indem Sie:
    - die Gewichte und deren Summe ausgeben (wie im Notebook [`4_OverUnderfitting.ipynb`](4_OverUnderfitting.ipynb)).
    - die Accuracy auf den Testdaten ermitteln. 

In [9]:
lasso_default = LogisticRegression(max_iter=10000, penalty='l1', C=1.0, solver = "liblinear")
lasso_default.fit(X_train, y_train)

lasso_weak_reg = LogisticRegression(max_iter=10000, penalty='l1', C=100, solver = "liblinear")
lasso_weak_reg.fit(X_train, y_train)

lasso_strong_reg = LogisticRegression(max_iter=10000, penalty='l1', C=0.01, solver = "liblinear")
lasso_strong_reg.fit(X_train, y_train)

# print the coefficients and the coreesponding features names vertically
print(pd.DataFrame({"coeff":lasso_default.coef_[0], "name":X_train.columns}).sort_values(by="coeff", ascending=False))
print(pd.DataFrame({"coeff":lasso_weak_reg.coef_[0], "name":X_train.columns}).sort_values(by="coeff", ascending=False))
print(pd.DataFrame({"coeff":lasso_strong_reg.coef_[0], "name":X_train.columns}).sort_values(by="coeff", ascending=False))

print("")

# print the absolute sum of the coefficients
print("Coefficients sum of default regularization: " + str(np.sum(np.abs(lasso_default.coef_))))
print("Coefficients sum of weak regularization: " + str(np.sum(np.abs(lasso_weak_reg.coef_))))
print("Coefficients sum of strong regularization: " + str(np.sum(np.abs(lasso_strong_reg.coef_))))

print("")

# print the accuracy
print("Accuracy of default regularization: %.2f%%" % (accuracy_score(y_test, lasso_default.predict(X_test)) * 100.0))
print("Accuracy of weak regularization: %.2f%%" % (accuracy_score(y_test, lasso_weak_reg.predict(X_test)) * 100.0))
print("Accuracy of strong regularization: %.2f%%" % (accuracy_score(y_test, lasso_strong_reg.predict(X_test)) * 100.0))

      coeff    name
3  0.297984   Parch
4  0.007300    Fare
1 -0.037649     Age
2 -0.308792   SibSp
0 -0.936572  Pclass
      coeff    name
3  0.326984   Parch
4  0.005965    Fare
1 -0.041726     Age
2 -0.336115   SibSp
0 -1.051465  Pclass
      coeff    name
4  0.016453    Fare
0  0.000000  Pclass
2  0.000000   SibSp
3  0.000000   Parch
1 -0.028366     Age

Coefficients sum of default regularization: 1.5882964601512035
Coefficients sum of weak regularization: 1.7622550929643404
Coefficients sum of strong regularization: 0.04481872316654524

Accuracy of default regularization: 69.23%
Accuracy of weak regularization: 68.53%
Accuracy of strong regularization: 66.43%


### Aufgabe 4. Cross-Validierung über  `C`
Ein guter Wert für `C` soll nun mit Hilfe der Methode `cross_val_score` gefunden werden. Ein Beispiel für diese Methode ist:

In [10]:
from sklearn.model_selection import cross_val_score

model = LogisticRegression(max_iter=1000, penalty = "l1", C=1.0, solver="liblinear")
model_cvs = cross_val_score(model, X_train, y_train, cv=5, scoring="accuracy")

Die Methode führt eine k-fold Cross-Validation auf den Trainingsdaten durch und gibt für jeden der k-folds den Score (in diesem Fall "accuracy") des Modell auf den jeweiligen Validierungsdaten zurück.

1. Berechnen Sie den Durchschnitt der Rückgabewerte von `cross_val_score` um für das gegebene `model` den durchschnittlichen Accuarcy-Score über alle k-folds zu erhalten.
2. Berechnen Sie für jeden der Werte $ C \in [0.1, 0.2, ...,9.8, 9.9]$ den durchschnittlichen Accuarcy-Score.
3. Trainieren Sie das Modell mit dem besten Wert für $C$ auf allen Trainingsdaten.
4. Bestimmen Sie die Accuracy dieses Modells auf den Testdaten.

In [11]:
# print the average accuracy
print("Average accuracy: " + str(np.mean(model_cvs)))

Average accuracy: 0.6988558352402746


In [12]:
# create a list of alphas from 0 to 10 with increments of 0.1
alphas = list(np.arange(0.1, 10, 0.1))
# create a list of models
models = [LogisticRegression(max_iter=1000, penalty = "l1", C=alpha, solver="liblinear") for alpha in alphas]


In [13]:
# print how many models we have
print("Number of models: " + str(len(models)))

Number of models: 99


In [14]:
cv_scores = [cross_val_score(model, X_train, y_train, cv=5, error_score='raise').mean() for model in models]
# find the best alpha
best_alpha = alphas[np.argmax(cv_scores)]
print("Best alpha: " + str(best_alpha))

Best alpha: 0.30000000000000004


In [15]:
# train the model with the best alpha
model = LogisticRegression(max_iter=1000, penalty = "l1", C=best_alpha, solver="liblinear")
model.fit(X_train, y_train)

LogisticRegression(C=0.30000000000000004, max_iter=1000, penalty='l1',
                   solver='liblinear')

In [16]:
# print the accuracy
print("Accuracy: %.2f%%" % (accuracy_score(y_test, model.predict(X_test)) * 100.0))

Accuracy: 69.23%
